In [1]:
import xarray as xr
import xclim
from distributed import Client
from dask import array
import zarr

In [2]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit="2GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51515,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:51529,Total threads: 2
Dashboard: http://127.0.0.1:51530/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:51518,


In [2]:
HURS_PATH = "/Users/ericcollins/climate-risk-map-hazards/Wildfire/FWI/input/hurs_day_CESM2_ssp126_r4i1p1f1_gn_2030_v1.1.nc"
PR_PATH = "/Users/ericcollins/climate-risk-map-hazards/Wildfire/FWI/input/pr_day_CESM2_ssp126_r4i1p1f1_gn_2030_v1.1.nc"
TAS_PATH = "/Users/ericcollins/climate-risk-map-hazards/Wildfire/FWI/input/tas_day_CESM2_ssp126_r4i1p1f1_gn_2030.nc"
SFC_WIND_PATH = "/Users/ericcollins/climate-risk-map-hazards/Wildfire/FWI/input/sfcWind_day_CESM2_ssp126_r4i1p1f1_gn_2030.nc"

In [ ]:
# Open the dataset
ds = xr.open_mfdataset(paths=[HURS_PATH, PR_PATH, TAS_PATH, SFC_WIND_PATH])

# Specify the rechunking - make the time dimension contiguous
target_chunks = {'time': -1, "lat": "auto", "lon": "auto"}  # Use -1 to make 'time' a single chunk
rechunked = ds.chunk(target_chunks)

# Save to Zarr for efficient rechunking
zarr_store = "output_rechunked.zarr"
rechunked.to_zarr(zarr_store, mode='w')

In [ ]:
# Open the dataset
ds = xr.open_mfdataset(paths=[HURS_PATH, PR_PATH, TAS_PATH, SFC_WIND_PATH])

# Specify the rechunking - make the time dimension contiguous
target_chunks = {'time': -1, "lat": 5, "lon": 12}  # Use -1 to make 'time' a single chunk
rechunked = ds.chunk(target_chunks)

# Save to Zarr for efficient rechunking
zarr_store = "output_rechunked_2.zarr"
rechunked.to_zarr(zarr_store, mode='w')

In [ ]:
# Open the dataset
ds = xr.open_mfdataset(paths=[HURS_PATH, PR_PATH, TAS_PATH, SFC_WIND_PATH])

# Specify the rechunking - make the time dimension contiguous
target_chunks = {'time': -1, "lat": 15, "lon": 36}  # Use -1 to make 'time' a single chunk
rechunked = ds.chunk(target_chunks)

# Save to Zarr for efficient rechunking
zarr_store = "output_rechunked_3.zarr"
rechunked.to_zarr(zarr_store, mode='w')

In [5]:
# Open the dataset
ds = xr.open_mfdataset(paths=[HURS_PATH, PR_PATH, TAS_PATH, SFC_WIND_PATH])

# Specify the rechunking - make the time dimension contiguous
target_chunks = {'time': -1, "lat": 25, "lon": 60}  # Use -1 to make 'time' a single chunk
rechunked = ds.chunk(target_chunks)

# Save to Zarr for efficient rechunking
zarr_store = "output_rechunked_4.zarr"
rechunked.to_zarr(zarr_store, mode='w')

2024-12-03 12:13:31,166 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle a831ca60cd0dc068307f40b713e731f9 initialized by task ('rechunk-merge-rechunk-transfer-rechunk-transfer-ddbee1915c244117e4481ee5e5076f3e', 0, 0, 0, 8, 0, 0) executed on worker tcp://127.0.0.1:51535
2024-12-03 12:13:34,844 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 82a19a54c5365c21cb80ac2f9575641c initialized by task ('rechunk-merge-rechunk-transfer-rechunk-transfer-0d0b1cf551b7a8b5a3776baa67052ce4', 0, 0, 0, 8, 0, 0) executed on worker tcp://127.0.0.1:51535
2024-12-03 12:13:40,550 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle a831ca60cd0dc068307f40b713e731f9 deactivated due to stimulus 'task-finished-1733256820.54395'
2024-12-03 12:13:43,193 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f22eb6aafe4c92b85b2e9b93f248ea1a initialized by task ('rechunk-merge-rechunk-transfer-rechunk-transfer-54a3cb26990afd4ab1671fb19f7425d8', 0, 0, 0, 8, 0, 0) executed on worker t

In [ ]:
# Combined dataset
ds = xr.open_zarr("output_rechunked.zarr")
out_fwi = xclim.indicators.atmos.cffwis_indices(
        tas=ds.tas,
        pr=ds.pr,
        hurs=ds.hurs,
        sfcWind=ds.sfcWind,
        lat=ds.lat,
        #season_mask=season_mask,
        season_method = None,
        #dry_start = None,
        #initial_start_up = False,
        overwintering=False,
    )

names = ['dc','dmc','ffmc','isi','bui','fwi']
da_out = xr.Dataset({name: da for name, da in zip(names, out_fwi)})
da_out.compute()

In [ ]:
(2 * 150 * 12 * 5) / 60

In [ ]:
import numpy as np

np.full_like(ds.tas, np.nan)



In [ ]:
ONE_YEAR_GLOBAL = 200
YEARS = 100


In [8]:
ds = xr.open_zarr("rechunked_31.zarr")

In [ ]:
ds.fwi.isel(time=200).compute().plot()

In [3]:
int(16/6)

2

In [3]:
ds = xr.open_mfdataset(paths=[HURS_PATH, PR_PATH, TAS_PATH, SFC_WIND_PATH])

In [6]:
ds.tas.chunks

((1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [12]:
ds.chunks

Frozen({'time': (365,), 'lat': (150, 150, 150, 150), 'lon': (360, 360, 360, 360)})